In [2]:
import time
import pandas as pd
from datetime import datetime
import requests
import json
import os


Here i tried to create a function to iterate through the dates which i needed to get extra data from, after i found an API with a more apropriate endpoint. Left it here for future reference. 

In [ ]:
#dates = pd.date_range(start="2021-06-09",end="2021-07-27")

In [ ]:
#dates

In [ ]:
"""dates = pd.date_range(start="2021-06-09",end="2021-07-27")
results = []

for mc in dates:
    res = requests.get(f"https://api.coingecko.com/api/v3/coins/binancecoin/history?date= {dates}/market_chart")

    if res.status_code == 200:
        results.append(res['market_data']['market_cap']['usd'].txt)
    else:
        print("Error in call")
    time.sleep(2)"""

## Request del 9-11-2017 hasta 27-07-2021

After finding the correct endpoint for my API, i was able to call it and receive the information i needed. This was then converted into a JSON to make it more readable.

In [3]:
url_bnb = "https://api.coingecko.com/api/v3/coins/binancecoin/market_chart/range?vs_currency=usd&from=1510185600&to=1627344000000"
res2 = requests.get(url_bnb)

In [ ]:
#res2.content

In [4]:
datos = requests.get(url_bnb).json()

In [5]:
datos

{'prices': [[1510185600000, 2.034664110158284],
  [1510272000000, 1.8070169067430286],
  [1510358400000, 1.6915633452049705],
  [1510444800000, 1.524949741792881],
  [1510531200000, 1.6739537350179436],
  [1510617600000, 1.58246676881987],
  [1510704000000, 1.52243686388942],
  [1510790400000, 1.59569777786549],
  [1510876800000, 1.51454613875746],
  [1510963200000, 1.67240752925033],
  [1511049600000, 1.60129507521577],
  [1511136000000, 1.62070853606007],
  [1511222400000, 1.57171927559821],
  [1511308800000, 1.60984260741505],
  [1511395200000, 1.55999317681984],
  [1511481600000, 1.69102104673283],
  [1511568000000, 1.85055461611236],
  [1511654400000, 1.84320442928677],
  [1511740800000, 1.96992379425731],
  [1511827200000, 1.98889715454395],
  [1511913600000, 1.78805621528696],
  [1512000000000, 2.00325216804839],
  [1512086400000, 2.09348995169993],
  [1512172800000, 2.11889373830539],
  [1512259200000, 2.26290697069717],
  [1512345600000, 2.57452340590756],
  [1512432000000, 2.

I checked the length of the data i wanted to extract from my JSON. I did this to make sure i had enough data for both fields.

In [6]:
len(datos['market_caps'])



1392

In [7]:
len(datos["total_volumes"])

1392

## Df timestamps & market cap

When i received the data, i noticed that the date was in unix timestamp, i checked the lists to make sure that they were in the correct order and iterated through them to be able to extract to data from JSON to list form

In [11]:
mc= []

for i in datos['market_caps']:
    mc.append(i)
    

In [12]:
#mc

Turning the market cap data into a dataframe

In [19]:
df = pd.DataFrame(mc)

In [28]:
df.head()

,unix_timestamp,market_cap
0,1510185600000,2.014602e+08
1,1510272000000,1.789200e+08
2,1510358400000,1.674885e+08
3,1510444800000,1.509914e+08
4,1510531200000,1.657449e+08


In [21]:
df = df.rename(columns={0:"unix_timestamp", 1:"market_cap"})

In [27]:
df.head()

,unix_timestamp,market_cap
0,1510185600000,2.014602e+08
1,1510272000000,1.789200e+08
2,1510358400000,1.674885e+08
3,1510444800000,1.509914e+08
4,1510531200000,1.657449e+08


Changing the name of the columns from indexes to actual names to make more sense, also important for the merge later on

## DF of unix timestamps and total volumns

More or less the same procedure as with the market cap data, iterating through the list of values and turning them into a dataframe. Also renaming the columns for future reference

In [13]:
tv= []

for i in datos['total_volumes']:
    tv.append(i)
    

In [14]:
#tv

In [15]:
df2 = pd.DataFrame(tv)


In [16]:
df2 = df2.rename(columns={0:"unix_timestamp", 1:"total_volume"})

In [26]:
df2.head()

,unix_timestamp,total_volume
0,1510185600000,1.136725e+07
1,1510272000000,7.770039e+06
2,1510358400000,5.531034e+06
3,1510444800000,9.288921e+06
4,1510531200000,6.612122e+06


A new DataFrame is created when the inner join is made between the previous dataframes. Inner join was used here to take the coincident column as "index"

In [23]:
result = pd.merge(df, df2, how="inner", on=["unix_timestamp"])

In [25]:
result.head()

,unix_timestamp,market_cap,total_volume
0,1510185600000,2.014602e+08,1.136725e+07
1,1510272000000,1.789200e+08,7.770039e+06
2,1510358400000,1.674885e+08,5.531034e+06
3,1510444800000,1.509914e+08,9.288921e+06
4,1510531200000,1.657449e+08,6.612122e+06


Renaming of columns due to names after join not being the precise ones.

In [29]:
result = result.rename(columns={"market_cap_x":"market_cap", "market_cap_y":"total_volume"})

In [31]:
result.head()

,unix_timestamp,market_cap,total_volume
0,1510185600000,2.014602e+08,1.136725e+07
1,1510272000000,1.789200e+08,7.770039e+06
2,1510358400000,1.674885e+08,5.531034e+06
3,1510444800000,1.509914e+08,9.288921e+06
4,1510531200000,1.657449e+08,6.612122e+06


New column created to change the unix timestamps into actual dates, to make the dataframe clearer and more understandable to guests

In [32]:
result['date'] = pd.to_datetime(result['unix_timestamp'], unit="ms")

In [34]:
result.head()

,unix_timestamp,market_cap,total_volume,date
0,1510185600000,2.014602e+08,1.136725e+07,2017-11-09
1,1510272000000,1.789200e+08,7.770039e+06,2017-11-10
2,1510358400000,1.674885e+08,5.531034e+06,2017-11-11
3,1510444800000,1.509914e+08,9.288921e+06,2017-11-12
4,1510531200000,1.657449e+08,6.612122e+06,2017-11-13


Dropping of timestamp column as not necessary due to new date column being more understandable

In [35]:
result.drop(columns=['unix_timestamp'], inplace= True)

In [37]:
result.head()

,market_cap,total_volume,date
0,2.014602e+08,1.136725e+07,2017-11-09
1,1.789200e+08,7.770039e+06,2017-11-10
2,1.674885e+08,5.531034e+06,2017-11-11
3,1.509914e+08,9.288921e+06,2017-11-12
4,1.657449e+08,6.612122e+06,2017-11-13


Renaming dataframe

In [38]:
df3= result[["date", "market_cap", "total_volume"]]

In [40]:
df3.head()

,date,market_cap,total_volume
0,2017-11-09,2.014602e+08,1.136725e+07
1,2017-11-10,1.789200e+08,7.770039e+06
2,2017-11-11,1.674885e+08,5.531034e+06
3,2017-11-12,1.509914e+08,9.288921e+06
4,2017-11-13,1.657449e+08,6.612122e+06


Here we noticed that the API call had called one months extra data in comparisons with the original dataframe we can created from .csv, so we dropped those fields as they were unecessary and would create a problem with merging of dataframes

In [41]:
n= 33

In [42]:
df3.drop(df3.tail(n).index,inplace = True)

/home/lukesmith194/.local/lib/python3.8/site-packages/pandas/core/frame.py:4901: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [43]:
df3.rename(columns={"date":"Date"}, inplace= True)

/home/lukesmith194/.local/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [45]:
df3.head()

,Date,market_cap,total_volume
0,2017-11-09,2.014602e+08,1.136725e+07
1,2017-11-10,1.789200e+08,7.770039e+06
2,2017-11-11,1.674885e+08,5.531034e+06
3,2017-11-12,1.509914e+08,9.288921e+06
4,2017-11-13,1.657449e+08,6.612122e+06


## Importing of original dataframe with pickle option

Pickle importation method was chosen instead of typical csv as the date column lost its format, changing from datestamp to object.

In [46]:
dfbnb = pd.read_pickle("data/bnb_original.pkl")

In [48]:
dfbnb.drop(["Unnamed: 0"] ,axis= 1, inplace= True)

In [50]:
dfbnb.head()

,Date,"Price($,mean)",High,Low,Daily_diff($),Change%,Vol.
0,2017-11-09,1.99,2.20,1.90,0.30,0.00,283.35K
1,2017-11-10,1.71,2.02,1.60,0.42,-14.07,483.23K
2,2017-11-11,1.66,1.93,1.58,0.35,-2.92,256.71K
3,2017-11-12,1.49,1.70,1.34,0.36,-10.24,456.48K
4,2017-11-13,1.68,1.74,1.49,0.25,12.75,166.01K


After importing and joining the dataframes, various changes in name and dropping of duplicated columns where made to make the dataframe more user friendly

In [51]:
bnb_final2= dfbnb.join(df3, lsuffix= "dfbnb", rsuffix="df3")

In [ ]:
#bnb_final = pd.merge(dfbnb, df3, on="Date")

In [ ]:
#bnb_final

In [ ]:
#bnb_final2.isna().sum()

In [52]:
bnb_final2.drop(columns=['Datedf3'], inplace= True)

In [53]:
bnb_final2.rename(columns={"Datedfbnb": "Date"}, inplace= True)

In [55]:
bnb_final2.head()

,Date,"Price($,mean)",High,Low,Daily_diff($),Change%,Vol.,market_cap,total_volume
0,2017-11-09,1.99,2.20,1.90,0.30,0.00,283.35K,2.014602e+08,1.136725e+07
1,2017-11-10,1.71,2.02,1.60,0.42,-14.07,483.23K,1.789200e+08,7.770039e+06
2,2017-11-11,1.66,1.93,1.58,0.35,-2.92,256.71K,1.674885e+08,5.531034e+06
3,2017-11-12,1.49,1.70,1.34,0.36,-10.24,456.48K,1.509914e+08,9.288921e+06
4,2017-11-13,1.68,1.74,1.49,0.25,12.75,166.01K,1.657449e+08,6.612122e+06


### A function was created here to deal with the Vol. column. It was not considered as an integer as the numbers had and additional K or M depending on if it was in millions or thousands.

The function passes a string, which was found in the dataframe at this time and if a K is found, it returns the number as a float. In the case of the millions it does the same with the M, but also multiplies the number by a thousand so that the number would be representative of its correct value

In [56]:
def km2float(s):
    S = s.upper()
    if 'K' in S:
        return float(s[:-1])
    elif 'M' in S:
        return float(s[:-1])*1000
    else:
        return 0

In [57]:
bnb_final2["Vol_K"] = bnb_final2["Vol."].apply(km2float)

In [58]:
bnb_final2.head()

,Date,"Price($,mean)",High,Low,Daily_diff($),Change%,Vol.,market_cap,total_volume,Vol_K
0,2017-11-09,1.99,2.20,1.90,0.30,0.00,283.35K,2.014602e+08,1.136725e+07,283.35
1,2017-11-10,1.71,2.02,1.60,0.42,-14.07,483.23K,1.789200e+08,7.770039e+06,483.23
2,2017-11-11,1.66,1.93,1.58,0.35,-2.92,256.71K,1.674885e+08,5.531034e+06,256.71
3,2017-11-12,1.49,1.70,1.34,0.36,-10.24,456.48K,1.509914e+08,9.288921e+06,456.48
4,2017-11-13,1.68,1.74,1.49,0.25,12.75,166.01K,1.657449e+08,6.612122e+06,166.01


Drops duplicate column for more useful column after the function is passed

In [59]:
bnb_final2.drop(["Vol."] ,axis= 1, inplace= True)

Finally exported to Pickle format

In [60]:
bnb_final2.to_pickle(r'bnb_final.pkl')